In [ ]:
#| default_exp arena_agent
#| test: false


"Chat UI + LLM agent that talks to FastMCP tools."


'Chat UI + LLM agent that talks to FastMCP tools.'

In [ ]:
#| export
#| eval: false

from __future__ import annotations
import asyncio, html, json, os
from typing import AsyncIterator, List, Dict

from fastapi import FastAPI, Request, status
from fastapi.responses import HTMLResponse
from fasthtml import FastHTML
from fasthtml.common import Div, Form, Input, Button, H1
from sse_starlette.sse import EventSourceResponse


from DataTalks.parking_tools import parking_status, nearest_parking_ids
from agents import Agent, Runner
from agents.mcp import MCPServerSse


In [ ]:
#| export
#| eval: false

# ── Config (env-vars for docker-compose) ───────────────────────────────────
MCP_URL        = os.getenv("MCP_URL", "http://tools:9001/sse")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")


In [ ]:
#| export
#| eval: false

# ── FastHTML shell with Tailwind + HTMX + SSE ext ─────────────────────────
app_html = FastHTML(
    hdrs=(
        '<script src="https://cdn.tailwindcss.com"></script>',
        '<script src="https://cdn.jsdelivr.net/npm/daisyui@4/dist/full.min.js"></script>',
        '<script src="https://unpkg.com/htmx.org@1.9.10/dist/htmx.min.js"></script>',
        '<script src="https://unpkg.com/htmx-ext-sse@2.2.3/dist/sse.js"></script>',
    ),
    live=True,
)

# FastAPI wrapper so uvicorn can find the ASGI app
app = FastAPI(title="Arena Buddy", docs_url=None)
app.mount("/", app_html)


In [ ]:
#| export
#| eval: false

# ── In-memory chat log ────────────────────────────────────────────────────
MSG: List[Dict[str, str]] = []
MSG_LOCK = asyncio.Lock()


In [ ]:
#| export
#| eval: false

# ── UI helpers ─────────────────────────────────────────────────────────────
def _chat_bubble(idx: int, **hx):
    role, txt = MSG[idx]["role"], MSG[idx]["content"] or "…"
    side   = "chat-end" if role == "user" else "chat-start"
    bubble = "bg-sky-700 text-white" if role == "assistant" else "bg-gray-200"
    return Div(
        Div(role, cls="chat-header text-xs text-gray-500"),
        Div(txt if role == "user" else html.unescape(txt),
            cls=f"chat-bubble {bubble}", **hx),
        cls=f"chat {side}", id=f"m{idx}",
    )

def _chat_input():
    return Input(
        id="msgin", name="msg", type="text", autocomplete="off",
        placeholder="Type your question…",
        cls="input input-bordered w-full",
        hx_swap_oob="true",
        onkeyup="event.key==='Enter' && this.form.requestSubmit()",
    )


In [ ]:
#| export
#| eval: false

# ── Home page ─────────────────────────────────────────────────────────────
@app_html.get("/")
async def home():
    ui = Div(
        H1("Arena Buddy", cls="text-3xl font-bold mb-4"),
        Div(id="chatlog",
            cls="space-y-3 mb-4 h-[70vh] overflow-y-auto bg-base-200 p-4 rounded-box"),
        Form(
            Div(_chat_input(),
                Button("Send ✈", cls="btn btn-primary ml-2"),
                cls="flex"),
            hx_post="/send", hx_target="#chatlog", hx_swap="beforeend",
        ),
        cls="max-w-2xl mx-auto p-6",
    )
    return ui             # FastHTML wraps it with <html> & scripts


In [ ]:
#| export
#| eval: false

# ── LLM helper ─────────────────────────────────────────────────────────────
async def _assistant_html(prompt: str) -> str:
    async with MCPServerSse(name="ui", params={"url": MCP_URL}) as srv:
        agent = Agent(
            "assistant",
            instructions=(
                "You help users reach Metro Areena Espoo with the "
                "lowest-emission mode possible. Use provided tools."
            ),
            mcp_servers=[srv]    
        )
        res = await Runner.run(starting_agent=agent, input=prompt)
        return res.final_output


In [ ]:
#| export
#| eval: false

# ── /send endpoint ────────────────────────────────────────────────────────
@app_html.post("/send")
async def send(request: Request):
    form   = await request.form()
    prompt = str(form.get("msg", "")).strip()
    if not prompt:
        return HTMLResponse("", status_code=status.HTTP_204_NO_CONTENT)

    async with MSG_LOCK:
        MSG.extend([
            {"role": "user",      "content": html.escape(prompt)},
            {"role": "assistant", "content": ""},
        ])
        idx_user, idx_asst = len(MSG) - 2, len(MSG) - 1

    return (
        _chat_bubble(idx_user).__html__() +
        _chat_bubble(
            idx_asst,
            hx_ext="sse",
            sse_connect=f"/stream/{idx_asst}",
            sse_swap="message",
            sse_close="close",
            hx_swap="innerHTML",
        ).__html__() +
        _chat_input().__html__()
    )


In [ ]:
#| export
#| eval: false

# ── /stream/{idx} endpoint (SSE) ──────────────────────────────────────────
@app_html.get("/stream/{idx}")
async def stream(idx: int):
    async def gen() -> AsyncIterator[str]:
        prompt_html = MSG[idx - 1]["content"]            # previous user msg
        reply_html  = await _assistant_html(prompt_html)

        async with MSG_LOCK:
            MSG[idx]["content"] = reply_html

        data = json.dumps({"html": reply_html})
        yield f"event: message\ndata: {data}\n\n"
        yield "event: close\ndata:\n\n"

    return EventSourceResponse(gen())
